# Giskard interface in PyCRAM
This notebook should provide you with an example on how to use the Giskard interface. This includes how to use the interface, how it actually works and how to extend it. 

We start installing and starting Giskard. For the installation please follow the instructions [here](https://github.com/SemRoCo/giskardpy).
After you finish the installation you should be able to start giskard by calling 
```
roslaunch giskardpy giskardpy_pr2_standalone.launch
```

In this way you can start Giskard for any robot that is supported.
```
roslaunch giskardpy giskardpy_hsr_standalone.launch
```
"Standalone" means that Giskard only uses a simulated robot and does not look for a real robot. If you want to use Giskard with a real robot you have to switch "standalone" with "iai". 
```
roslaunch giskardpy giskardpy_hsr_iai.launch
```

To see what Giskard is doing you can start RViz, there should already be a MarkerArray when starting otherwise you have to add this manually.

## How to use the Giskard interface 
Everything related to the Giskard interface is located in ```pycram.external_interfaces.giskard```. 
The content of the file can be roughtly divided into three parts:
    1. Methods to manage the beliefe states between PyCRAM and Giskard
    2. Motion goals that should be send to Giskard for execution 
    3. Helper methods to construct ROS messages
    
The most useful methods are the ones for sending and executing Motion goals. These are the ones we will mostly look at.

We will now start by setting up PyCRAM and then try to send some simple motion goals.

In [1]:
from pycram.bullet_world import BulletWorld, Object
from pycram.enums import ObjectType

world = BulletWorld()

pybullet build time: Sep 20 2021 20:33:29
Unknown attribute "type" in /robot[@name='pr2']/link[@name='base_laser_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='wide_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='narrow_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='laser_tilt_link']
/home/bass/cram_ws/src/pycram/src/pycram/orm/action_designator.py:10: SAWarning: Implicitly combining column Designator.dtype with column Action.dtype under attribute 'dtype'.  Please configure one or more attributes for these same-named columns explicitly.
  class Action(MapperArgsMixin, Designator):
/home/bass/cram_ws/src/pycram/src/pycram/orm/motion_designator.py:16: SAWarning: Implicitly combining column Designator.dtype with column Motion.dtype under attribute 'dtype'.  Please configure one or more attributes for these same-named columns explicitly.
  class Motion(MapperArgsMixin, Designator):
[WARN] [1701081293

In [2]:
pr2 = Object("pr2", ObjectType.ROBOT, "pr2.urdf")

Unknown tag "material" in /robot[@name='plane']/link[@name='planeLink']/collision[1]
Unknown tag "contact" in /robot[@name='plane']/link[@name='planeLink']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='base_laser_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='wide_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='narrow_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='laser_tilt_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='base_laser_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='wide_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='narrow_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='laser_tilt_link']


When you are working on the real robot you also need to initialize the RobotStateUpdater, this module updates the robot in the BulletWorld with the pose and joint state of the real robot. 

You might need to change to topic names to fit the topic names as published by your robot. 

In [3]:
from pycram.ros.robot_state_updater import RobotStateUpdater
import rospy
from sensor_msgs.msg import JointState

r = RobotStateUpdater("/tf", "/joint_states")
joint_state_publisher = rospy.Publisher("/joint_states", JointState, queue_size=10, latch=True)


Now we have a PyCRAM belief state set up, belief state in this case just refeers to the BulletWorld since the BulletWorld represents what we belief the world to look like. 

The next step will be to send a simple motion goal. The motion goal we will be sending is moving the torso up. For this we just need to move one joint so we use the ```achive_joint_goal```. This method takes a dictionary with the joints that should be moved and the target value for the joint. 

Look at Rviz to see the robot move, since we call Giskard for movement the robot in the BulletWorld will not move.

In [5]:
from pycram.external_interfaces import giskard


giskard_result = giskard.achieve_joint_goal({"torso_lift_joint": 0.28})
joints = giskard_result.trajectory.joint_names
trajectory_points = giskard_result.trajectory.points

# publish the joint states to the robot state updater
if giskard_result.error_codes[-1] == giskard_result.SUCCESS:
    joint_states = JointState()
    joint_states.name = joints[3:]
    joint_states.position = trajectory_points[-1].positions[3:]
    joint_state_publisher.publish(joint_states)


For Giskard everything is connected by joints (this is called [World Tree](https://github.com/SemRoCo/giskardpy/wiki/World-Tree) by Giskard) therefore we can move the robot by using a motion goal between the map origin and the robot base. 

In the example below we use a cartesian goal, meaning we give Giskard a goal pose, a root link and a tip link and Giskard tries to move all joints between root link and tip link such that the tip link is at the goal pose.

This sort of movement is fine for short distance but keep in mind that Giskard has no collision avoidance for longer journeys. So using MoveBase for longer distances is a better idea.

In [6]:
from pycram.external_interfaces import giskard
from pycram.pose import Pose

giskard.achieve_cartesian_goal(Pose([1, 0, 0]), "pr2/base_link", "map")

error_codes: [0]
error_messages: 
  - None
trajectory: 
  header: 
    seq: 0
    stamp: 
      secs: 0
      nsecs:         0
    frame_id: ''
  joint_names: 
    - x
    - y
    - yaw
    - fl_caster_rotation_joint
    - fl_caster_l_wheel_joint
    - fl_caster_r_wheel_joint
    - fr_caster_rotation_joint
    - fr_caster_l_wheel_joint
    - fr_caster_r_wheel_joint
    - bl_caster_rotation_joint
    - bl_caster_l_wheel_joint
    - bl_caster_r_wheel_joint
    - br_caster_rotation_joint
    - br_caster_l_wheel_joint
    - br_caster_r_wheel_joint
    - torso_lift_joint
    - head_pan_joint
    - head_tilt_joint
    - laser_tilt_mount_joint
    - r_shoulder_pan_joint
    - r_shoulder_lift_joint
    - r_upper_arm_roll_joint
    - r_elbow_flex_joint
    - r_forearm_roll_joint
    - r_wrist_flex_joint
    - r_wrist_roll_joint
    - r_gripper_motor_slider_joint
    - r_gripper_motor_screw_joint
    - r_gripper_l_finger_joint
    - r_gripper_l_finger_joint
    - r_gripper_l_finger_joint
    - r

Now for the last example: we will move the gripper using full body motion controll. 

We will again use the cartesian goal, but now between "map" and "r_gripper_tool_frame". This will not only move the robot (because the chain between "map" and "base_link" as used in the previous example is also part of this chain) but also move the arm of the robot such that it reaches the goal pose.

In [7]:
from pycram.external_interfaces import giskard
from pycram.pose import Pose

giskard.achieve_cartesian_goal(Pose([1, 0.5, 0.7]), "r_gripper_tool_frame", "map")

error_codes: [0]
error_messages: 
  - None
trajectory: 
  header: 
    seq: 0
    stamp: 
      secs: 0
      nsecs:         0
    frame_id: ''
  joint_names: 
    - x
    - y
    - yaw
    - fl_caster_rotation_joint
    - fl_caster_l_wheel_joint
    - fl_caster_r_wheel_joint
    - fr_caster_rotation_joint
    - fr_caster_l_wheel_joint
    - fr_caster_r_wheel_joint
    - bl_caster_rotation_joint
    - bl_caster_l_wheel_joint
    - bl_caster_r_wheel_joint
    - br_caster_rotation_joint
    - br_caster_l_wheel_joint
    - br_caster_r_wheel_joint
    - torso_lift_joint
    - head_pan_joint
    - head_tilt_joint
    - laser_tilt_mount_joint
    - r_shoulder_pan_joint
    - r_shoulder_lift_joint
    - r_upper_arm_roll_joint
    - r_elbow_flex_joint
    - r_forearm_roll_joint
    - r_wrist_flex_joint
    - r_wrist_roll_joint
    - r_gripper_motor_slider_joint
    - r_gripper_motor_screw_joint
    - r_gripper_l_finger_joint
    - r_gripper_l_finger_joint
    - r_gripper_l_finger_joint
    - r

That conludes this example you can now close the BulletWorld by using the "exit" method.

In [ ]:
world.exit()

## How the Giskard interface works 
The PyCRAM interface to Giskard mostly relies on the Python interface that Giskard already proviedes ([tutorial](https://github.com/SemRoCo/giskardpy/wiki/Python-Interface) and the [source code](https://github.com/SemRoCo/giskardpy/blob/master/src/giskardpy/python_interface.py)). This inteface provides methods to achive motion goals and load things into the Giskard believe state. 

What PyCRAM with this does is: Synchronize the believe state of Giskard with the one of PyCRAM by loading the environment URDF in Giskard, this is done before any motion goal is send. Furthermore, the motion goals are wrapped in methods that use PyCRAM data types. 

You can also set collisions between different groups of links. By default Giskard avoids all collisions but for things like grasping an object you want to allow collisions of the gripper. The interface also the following colliion modes:
    * avoid_all_collisions
    * allow_self_collision
    * allow_gripper_collision
The collision mode can be set by calling the respective method, after calling the method the collision mode is valid for the next motion goal afterwards it default back to avoid_all_collisions.

There is a ```init_giskard_interface``` method which can be used as a decorator. This decorator should be used on all methods that access the giskard_wrapper, since it assures that the interface is working and checks if Giskard died or the imports for the giskard_msgs failed. 

## Extend the Giskard interface
At the moment the PyCRAM Giskard interface is mostly a wrapper around the Python interface provided by Giskard. If you want to extend the interface there are two ways:
   * Wrap more motion goals which are provided by the Python interface 
   * Design new Higher-Level motion goals by combining the motion goals already provided